# ***GRAPH DES DONNEES DE COMPTAGES***
> tests sur les données OTV

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct

import pandas as pd
import altair as alt

# 1.Analyse du type d'année distincte de comptage en fonction du type de poste

In [19]:
with ct.ConnexionBdd('local_otv_boulot') as c:
    rqt="select * from qualite.vue_represent_cpt_annee_before"
    df=pd.read_sql(rqt,c.sqlAlchemyConn)
grp=df.groupby(['tiertile', 'type_poste']).id_comptag.count().reset_index()

In [55]:
g1=alt.Chart(grp, title='répartitipon des comptage par type').mark_bar().encode(
    x='tiertile:O', 
    y=alt.Y('id_comptag:Q', stack='normalize', axis=alt.Axis(title='Nb de comptage')), 
    color='type_poste:N')
g2=alt.Chart(grp).mark_bar().encode(
    x='tiertile:O', 
    y=alt.Y('id_comptag:Q', axis=alt.Axis(title='Nb de comptage')), 
    color='type_poste:N')
g1 | g2

alt.HConcatChart(...)

In [59]:
g3=alt.Chart(grp, title='répartitipon des comptage par type').mark_bar().encode(
    x='type_poste:N', 
    y=alt.Y('id_comptag:Q', stack='normalize', axis=alt.Axis(title='Nb de comptage')), 
    color='tiertile:N')
g4=alt.Chart(grp).mark_bar().encode(
    x='type_poste:N', 
    y=alt.Y('id_comptag:Q', axis=alt.Axis(title='Nb de comptage')), 
    color='tiertile:N')
g3 | g4

alt.HConcatChart(...)

In [60]:
with ct.ConnexionBdd('local_otv_boulot') as c:
    rqt="select * from qualite.vue_represent_globale"
    df=pd.read_sql(rqt,c.sqlAlchemyConn)

In [103]:
grp_global=df.groupby(['note_represent_globale', 'type_poste']).id.count().reset_index().merge(
                      pd.DataFrame({'txt_note':['non representatif', 'faible', 'moyenne', 'bonne'],
                                     'num_note':[0,1,3,6]}), left_on='note_represent_globale',
                      right_on='num_note')

In [126]:
alt.Chart(grp_global, title=['Répartition des comptages selon la représentativité globale','et le type de poste'], width=300).mark_bar(width=40).encode(
    x=alt.X('txt_note:N', sort=['note_represent_globale'], axis=alt.Axis(title='Représentativité', labelAngle=315)), 
    y=alt.Y('id:Q',axis=alt.Axis(title='Nb de comptage')),
    color='type_poste')

alt.Chart(...)

In [127]:
alt.Chart(grp_global.assign(cst=1), title=['Répartition des comptages selon la représentativité globale'], width=300).mark_bar(width=40).encode(
    x=alt.X('cst', axis=alt.Axis(title=None, labelColor='white')), 
    y=alt.Y('id:Q',axis=alt.Axis(title='Nb de comptage'), stack='normalize'),
    color=alt.Color('txt_note:N',legend=alt.Legend(title='Représentativité')))

alt.Chart(...)

In [130]:
alt.Chart(grp_global, title=['Répartition des comptages selon la représentativité globale','et le type de poste'], width=300).mark_bar(width=40).encode(
    x=alt.X('type_poste:N', axis=alt.Axis(title='type de poste', labelAngle=315)), 
    y=alt.Y('id:Q',axis=alt.Axis(title='Nb de comptage'), stack='normalize'),
    color=alt.Color('txt_note:N',legend=alt.Legend(title='Représentativité')))

alt.Chart(...)

# 2.Analyse de l'exhaustivite des données

In [6]:
with ct.ConnexionBdd('local_otv_boulot') as c:
    rqt="select * from qualite.vue_exaust_globale"
    df=pd.read_sql(rqt,c.sqlAlchemyConn)

In [20]:
grp=df.groupby(['note_finale_txt', 'type_poste']).id_comptag.count().reset_index().merge(pd.DataFrame({'color_index':[1,2,3], 'note_finale_txt':['faible', 'moyenne', 'bonne']}), on='note_finale_txt')

In [21]:
grp

,note_finale_txt,type_poste,id_comptag,color_index
0,bonne,permanent,822,3
1,faible,permanent,1709,1
2,faible,ponctuel,3916,1
3,faible,tournant,2523,1
4,moyenne,permanent,5520,2
5,moyenne,ponctuel,3943,2
6,moyenne,tournant,8168,2


In [22]:
alt.Chart(grp, title=['Répartition des comptages selon l\'exhaustivité globale','et le type de poste'], width=300).mark_bar(width=40).encode(
    x=alt.X('type_poste:N', axis=alt.Axis(title='type de poste', labelAngle=315), sort=['ponctuel', 'tournant', 'permanent']), 
    y=alt.Y('id_comptag:Q',axis=alt.Axis(title='Nb de comptage'), stack='normalize'),
    color=alt.Color('note_finale_txt:N',legend=alt.Legend(title='Représentativité'), sort=['faible', 'moyenne', 'bonne']),
    order =alt.Order('color_index:Q', sort='ascending'))

alt.Chart(...)